In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
IMAGE_SIZE=[224,224]

In [3]:
train_path='chest_xray/train'
test_path='chest_xray/test'

In [4]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
# don't train existing weights
for layer in tqdm(vgg.layers):
  layer.trainable = False

100%|███████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 19037.69it/s]


In [6]:
  # useful for getting number of classes
folders = glob('chest_xray/train/*')

In [7]:
len(folders)

2

In [8]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [9]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [10]:
# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [11]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [13]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
training_set = train_datagen.flow_from_directory('chest_xray/train/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5216 images belonging to 2 classes.


In [15]:
test_set = test_datagen.flow_from_directory('chest_xray/test/',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 624 images belonging to 2 classes.


In [ ]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
 84/163 [==============>...............] - ETA: 54:39 - loss: 0.6948 - accuracy: 0.531 - ETA: 50:22 - loss: 0.6349 - accuracy: 0.718 - ETA: 46:44 - loss: 0.9877 - accuracy: 0.729 - ETA: 46:19 - loss: 1.0473 - accuracy: 0.718 - ETA: 44:30 - loss: 0.8875 - accuracy: 0.762 - ETA: 43:14 - loss: 0.8745 - accuracy: 0.724 - ETA: 42:07 - loss: 0.8388 - accuracy: 0.718 - ETA: 41:21 - loss: 0.7613 - accuracy: 0.750 - ETA: 40:42 - loss: 0.7041 - accuracy: 0.760 - ETA: 40:09 - loss: 0.6789 - accuracy: 0.759 - ETA: 39:43 - loss: 0.6603 - accuracy: 0.764 - ETA: 39:19 - loss: 0.6442 - accuracy: 0.765 - ETA: 38:56 - loss: 0.6298 - accuracy: 0.766 - ETA: 38:31 - loss: 0.5955 - accuracy: 0.781 - ETA: 38:10 - loss: 0.5793 - accuracy: 0.787 - ETA: 37:46 - loss: 0.5784 - accuracy: 0.785 - ETA: 37:24 - loss: 0.5660 - accuracy: 0.790 - ETA: 37:01 - loss: 0.5504 - accuracy: 0.796 - ETA: 36:40 - loss: 0.5320 - accuracy: 0.801 - ETA: 36:18 - loss: 0.5177

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
r.history

In [ ]:
# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
import tensorflow as tf
from keras.models import load_model
model.save('facefeatures_new_model.h5')